
# Food‑11 × YOLO11 — Training Notebook (v2, patched)

This version fixes the Ultralytics classification **data path** issue by:
1) Creating `train/`, `val/`, `test` symlinks that point to your `training/`, `validation/`, `evaluation/` folders.
2) Passing `data=DATASET_ROOT` (a **directory**) to `YOLO.train` for classification.



In [1]:

# --- 1) Paths & switches -------------------------------------------------------
import os, pathlib
DATASET_ROOT = "/home/kristoffel/datasets/dataset-03-Food-11"
TRAIN_DIR    = os.path.join(DATASET_ROOT, "training")
VAL_DIR      = os.path.join(DATASET_ROOT, "validation")
TEST_DIR     = os.path.join(DATASET_ROOT, "evaluation")

MODEL_DIR    = "/home/kristoffel/models"
os.makedirs(MODEL_DIR, exist_ok=True)

TASK = "classify"   # 'classify' (Food-11) or 'detect' (needs YOLO labels)

RUN_NAME_CLS = "food11_yolo11n_cls"
RUN_NAME_DET = "food11_yolo11n_det"

CLASS_WEIGHTS = "yolo11n-cls.pt"                     # auto-downloads if not present
DET_WEIGHTS   = os.path.join(MODEL_DIR, "yolo11n.pt") # your base detection weights

for p in (TRAIN_DIR, VAL_DIR, TEST_DIR, MODEL_DIR):
    assert os.path.isdir(p), f"Missing directory: {p}"
print("✅ Paths OK. Task:", TASK)


✅ Paths OK. Task: classify


In [2]:

# --- 2) Install/Import Ultralytics & env info ---------------------------------
# !pip install -U ultralytics

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import torch
from ultralytics import YOLO
import ultralytics
print("Ultralytics:", ultralytics.__version__)
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))


Ultralytics: 8.3.203
PyTorch: 2.2.0a0+81ea7a4
CUDA available: True
Device: Tesla V100-SXM3-32GB


In [3]:

# --- 3) Create train/val/test symlinks for classification ---------------------
# Ultralytics classify expects a directory with 'train','val','test' subfolders.
# Your dataset uses 'training','validation','evaluation'. We'll create symlinks
# so we don't have to duplicate data or rename anything.
import pathlib, shutil

mapping = {"train":"training", "val":"validation", "test":"evaluation"}
for dst_name, src_name in mapping.items():
    dst = pathlib.Path(DATASET_ROOT) / dst_name
    src = pathlib.Path(DATASET_ROOT) / src_name
    if not src.is_dir():
        print(f"⚠️ Source split missing: {src}")
        continue
    if dst.exists():
        if dst.is_symlink() and dst.resolve() == src.resolve():
            print(f"✅ Symlink already exists: {dst} -> {src}")
            continue
        else:
            print(f"ℹ️ {dst} exists; leaving as-is.")
            continue
    try:
        dst.symlink_to(src)
        print(f"🔗 Created symlink: {dst} -> {src}")
    except Exception as e:
        # Fallback: copy if symlinks are not allowed on this filesystem.
        shutil.copytree(src, dst, dirs_exist_ok=True)
        print(f"📁 Copied {src} -> {dst} (symlink fallback)")


🔗 Created symlink: /home/kristoffel/datasets/dataset-03-Food-11/train -> /home/kristoffel/datasets/dataset-03-Food-11/training
🔗 Created symlink: /home/kristoffel/datasets/dataset-03-Food-11/val -> /home/kristoffel/datasets/dataset-03-Food-11/validation
🔗 Created symlink: /home/kristoffel/datasets/dataset-03-Food-11/test -> /home/kristoffel/datasets/dataset-03-Food-11/evaluation


In [4]:

# --- 4) (Optional) Build a YAML for reference (not used by classify train) ----
# We keep this for documentation and later testing/evaluation convenience.
import yaml, os
classes = sorted([d.name for d in pathlib.Path(TRAIN_DIR).iterdir() if d.is_dir()])
cls_yaml_path = os.path.join(MODEL_DIR, "food11-cls.yaml")
with open(cls_yaml_path, "w") as f:
    yaml.safe_dump({"train": TRAIN_DIR, "val": VAL_DIR, "test": TEST_DIR, "names": classes}, f, sort_keys=False)
print("Wrote:", cls_yaml_path)
print(open(cls_yaml_path).read())


Wrote: /home/kristoffel/models/food11-cls.yaml
train: /home/kristoffel/datasets/dataset-03-Food-11/training
val: /home/kristoffel/datasets/dataset-03-Food-11/validation
test: /home/kristoffel/datasets/dataset-03-Food-11/evaluation
names:
- Bread
- Dairy product
- Dessert
- Egg
- Fried food
- Meat
- Noodles-Pasta
- Rice
- Seafood
- Soup
- Vegetable-Fruit



In [5]:

# --- 5) Train — Classification -------------------------------------------------
if TASK != "classify":
    print("Skipping because TASK != 'classify'")
else:
    EPOCHS = 50   # ← edit me
    IMGSZ  = 224  # ← edit me
    BATCH  = 64   # ← edit me
    model = YOLO(CLASS_WEIGHTS)  # yolo11n-cls.pt

    results = model.train(
        data=DATASET_ROOT,     # NOTE: pass the directory (with train/val/test) not the YAML
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH,
        lr0=1e-3,
        patience=20,
        project=MODEL_DIR,
        name=RUN_NAME_CLS,
        plots=True,
        device=0 if torch.cuda.is_available() else "cpu",
    )
    print("Training complete. Best:", os.path.join(MODEL_DIR, RUN_NAME_CLS, "weights", "best.pt"))


New https://pypi.org/project/ultralytics/8.3.222 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.203 🚀 Python-3.10.12 torch-2.2.0a0+81ea7a4 CUDA:0 (Tesla V100-SXM3-32GB, 32494MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/kristoffel/datasets/dataset-03-Food-11, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fo

In [6]:

# --- 6) Validate on val and evaluation (as test) -------------------------------
if TASK != "classify":
    print("Skipping because TASK != 'classify'")
else:
    model_path = os.path.join(MODEL_DIR, RUN_NAME_CLS, "weights", "best.pt")
    model = YOLO(model_path if os.path.isfile(model_path) else CLASS_WEIGHTS)
    IMGSZ = 224

    metrics_val = model.val(
        data=DATASET_ROOT,  # directory with train/val/test
        split="val",
        imgsz=IMGSZ,
        project=MODEL_DIR,
        name=RUN_NAME_CLS + "_val",
        device=0 if torch.cuda.is_available() else "cpu",
    )
    metrics_test = model.val(
        data=DATASET_ROOT,
        split="test",
        imgsz=IMGSZ,
        project=MODEL_DIR,
        name=RUN_NAME_CLS + "_test",
        device=0 if torch.cuda.is_available() else "cpu",
    )
    print("Done. See runs in:", MODEL_DIR)


Ultralytics 8.3.203 🚀 Python-3.10.12 torch-2.2.0a0+81ea7a4 CUDA:0 (Tesla V100-SXM3-32GB, 32494MiB)
YOLO11n-cls summary (fused): 47 layers, 2,807,024 parameters, 0 gradients, 4.2 GFLOPs
ERROR ❌ train: /home/kristoffel/datasets/dataset-03-Food-11/train... found 9867 images in 12 classes (requires 11 classes, not 12)
val: /home/kristoffel/datasets/dataset-03-Food-11/val... found 3430 images in 11 classes ✅ 
test: /home/kristoffel/datasets/dataset-03-Food-11/test... found 3347 images in 11 classes ✅ 
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1035.1±569.5 MB/s, size: 44.3 KB)
val: Scanning /home/kristoffel/datasets/dataset-03-Food-11/val... 3430 images, 0 corrupt: 100% ━━━━━━━━━━━━ 3430/3430 3.5Mit/s 0.0s0s
val: /home/kristoffel/datasets/dataset-03-Food-11/val/Dairy product/6.jpg: corrupt JPEG restored and saved
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 215/215 49.6it/s 4.3s0.1s
                   all          0   0.000292
Speed: 0.1ms preprocess, 0.4ms inferen


## Notes
- You can safely ignore the optional YAML during training; it's here for record‑keeping.
- If you prefer to **rename** your folders to `train/`, `val/`, `test`, you can remove the symlinks and rename once.
- For detection, switch `TASK = "detect"` and ensure YOLO `labels/` exist (not typical for Food‑11).
